## 多个 feas_iti 且最后一趟列车 id 相同的乘客

|           |     rid | iti_id |    path_id | seg_id | train_id | board_ts | alight_ts |
| :-------- | ------: | -----: | ---------: | -----: | -------: | -------: | --------: |
| 93194730  | 1525794 |      2 | 1076103701 |      2 | 10301630 |    65071 |     65670 |
| 8610761   |  119014 |      6 | 1101102701 |      3 | 11002833 |    30045 |     30791 |
| 18578209  |  230428 |      6 | 1053108503 |      2 | 10401849 |    29715 |     30255 |
| 72855273  | 1321497 |      3 | 1110112802 |      1 | 10401844 |    60803 |     61537 |
| 105488240 | 1781873 |      2 | 1028106602 |      2 | 10401738 |    69396 |     69625 |
| 90908613  | 1485498 |      2 | 1023112302 |      2 | 10702370 |    64374 |     64780 |
| 91388272  | 1494286 |      1 | 1104111501 |      2 | 10702247 |    65070 |     65278 |
| 53621559  |  957712 |      6 | 1121102503 |      3 | 10401786 |    49174 |     50141 |
| 19767473  |  243641 |      1 | 1081103901 |      2 | 10702323 |    29509 |     31141 |
| 107978876 | 1880069 |      2 | 1129107002 |      1 | 10201226 |    73278 |     74154 |
| 106615427 | 1817923 |      1 | 1027110801 |      2 | 10301362 |    71471 |     73514 |
| 76737269  | 1359494 |      1 | 1099102101 |      2 | 10301421 |    64071 |     65266 |
| 100422001 | 1661221 |      5 | 1061108702 |      1 | 10100270 |    66558 |     66886 |
| 2408620   |   41409 |      2 | 1111106502 |      1 | 10401886 |    25727 |     25836 |
| 101105912 | 1675451 |      1 | 1110101301 |      1 | 10401914 |    66944 |     67031 |
| 92513072  | 1514419 |      3 | 1097110302 |      3 | 10200839 |    65302 |     65408 |
| 81121188  | 1412555 |      1 | 1061109504 |      2 | 10201199 |    63371 |     63480 |
| 108069499 | 1886032 |      1 | 1032108201 |      1 | 10200950 |    73568 |     75192 |
| 73737366  | 1331466 |      1 | 1086110901 |      2 | 10702414 |    61616 |     62107 |
| 48046191  |  811513 |      3 | 1076103602 |      3 | 10301265 |    42747 |     43084 |


In [1]:
from src import config
from src.utils import *
from src.globals import *
from src.itinerary import *

config.load_config()

wtd = WalkTimeDisModel(etd=get_etd(), ttd=get_ttd())

left = read_(config.CONFIG["results"]["left"], show_timer=False)
print(left.shape)
print(left.drop_duplicates("rid").shape)
print(left.drop_duplicates(["rid", "iti_id"]).shape)
left

[INFO] Reading file: results\egress\etd_1.csv
[INFO] Reading file: results\transfer\ttd_1.csv
[INFO] Reading file: results\network\pathvia.pkl
[INFO] Reading file: results\network\platform.csv
[INFO] Reading file: results\trajectory\left.pkl
(46637884, 7)
(1195446, 7)
(16339032, 7)


,rid,iti_id,path_id,seg_id,train_id,board_ts,alight_ts
2,285,1,1120101001,1,10702170,21132,22747
3,285,2,1120101001,1,10702170,25273,26962
4,285,3,1120101001,1,10702186,21797,23422
5,285,4,1120101001,1,10702186,25949,27655
6,285,5,1120101001,1,10702207,22652,24277
...,...,...,...,...,...,...,...
109949952,2046865,2,1097101501,1,10100645,85287,85823
109949955,2046868,1,1097101501,1,10100421,84945,85481
109949956,2046868,2,1097101501,1,10100645,85287,85823
109949965,2046883,1,1129101001,1,10702289,85531,85741


In [2]:
# Calculate entry walk distribution for all itineraries in `left`
print("[INFO] Calculating entry walk distribution for all itineraries...")
df_ent = cal_entry_dis_all(wtd=wtd, left=left)
df_ent["seg_id"] = 0  # set entry seg_id to 0

# Calculate egress walk distribution for all itineraries in `left`
print("[INFO] Calculating egress walk distribution for all itineraries...")
df_egr = cal_egress_dis_all(wtd=wtd, left=left)
df_egr["seg_id"] = -1  # set egress seg_id to -1

# Calculate transfer walk distribution for all itineraries in `left`
print("[INFO] Calculating transfer walk distribution for all itineraries...")
df_trans = cal_transfer_dis_all(wtd=wtd, left=left)

[INFO] Calculating entry walk distribution for all itineraries...
[INFO] Reading file: data\AFC.pkl
[INFO] Calculating egress walk distribution for all itineraries...
[INFO] Calculating transfer walk distribution for all itineraries...


In [3]:
print(df_ent.columns, df_egr.columns, df_trans.columns)

Index(['rid', 'iti_id', 'path_id', 'board_ts', 'entry_pp_id', 'ts1',
       'entry_time', 'dis', 'seg_id'],
      dtype='object') Index(['rid', 'iti_id', 'path_id', 'alight_ts', 'egress_pp_id', 'ts2',
       'egress_time', 'dis', 'seg_id'],
      dtype='object') Index(['rid', 'iti_id', 'path_id', 'seg_id', 'alight_ts', 'board_ts',
       'board_ts', 'transfer_time', 'transfer_type', 'pp_id_min', 'pp_id_max',
       'dis'],
      dtype='object')


In [7]:
# Calculate entry walk distribution for all itineraries in `left`
print("[INFO] Calculating entry walk distribution for all itineraries...")
df_ent = cal_entry_dis_all(wtd=wtd, left=left)
df_ent["seg_id"] = 0  # set entry seg_id to 0
df_ent = df_ent.rename(
    columns={
        "ts1": "t1",
        "board_ts": "t2",
        "entry_time": "time",
        "entry_pp_id": "pp_id1"}
)[[
    "rid", 'iti_id', 'path_id', 'seg_id', 't1', 't2', 'pp_id1', 'time', 'dis'
]]

# Calculate egress walk distribution for all itineraries in `left`
print("[INFO] Calculating egress walk distribution for all itineraries...")
df_egr = cal_egress_dis_all(wtd=wtd, left=left)
df_egr["seg_id"] = -1  # set egress seg_id to -1
df_egr = df_egr.rename(
    columns={
        "alight_ts": "t1",
        "ts2": "t2",
        "egress_time": "time",
        "egress_pp_id": "pp_id2",
    }
)[[
    "rid", "iti_id", "path_id", "seg_id", "t1", "t2", "pp_id2", "time", "dis"
]]

# Calculate transfer walk distribution for all itineraries in `left`
print("[INFO] Calculating transfer walk distribution for all itineraries...")
df_trans = cal_transfer_dis_all(wtd=wtd, left=left)
df_trans = df_trans.rename(
    columns={
        "pp_id_min": "pp_id1",
        "pp_id_max": "pp_id2",
        "transfer_time": "time",
        "alight_ts": "t1",
        "board_ts": "t2"
    }
)[[
    "rid", "iti_id", "path_id", "seg_id", "t1", "t2", "pp_id1", "pp_id2", "time", "dis"
]]

df = pd.concat([df_ent, df_egr, df_trans], ignore_index=True)
df

[INFO] Calculating entry walk distribution for all itineraries...
[INFO] Calculating egress walk distribution for all itineraries...
[INFO] Calculating transfer walk distribution for all itineraries...


InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [ ]:
df_ent.info()
df_egr.info()
df_trans.info()
pd.concat([df_ent, df_egr, df_trans], join="outer", ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
Index: 16339032 entries, 2 to 109949966
Data columns (total 9 columns):
 #   Column   Dtype  
---  ------   -----  
 0   rid      int32  
 1   iti_id   int32  
 2   path_id  int32  
 3   seg_id   int64  
 4   t1       int64  
 5   t2       int32  
 6   pp_id1   int64  
 7   time     int64  
 8   dis      float64
dtypes: float64(1), int32(4), int64(4)
memory usage: 1.5 GB
<class 'pandas.core.frame.DataFrame'>
Index: 16339032 entries, 2 to 109949966
Data columns (total 9 columns):
 #   Column   Dtype  
---  ------   -----  
 0   rid      int32  
 1   iti_id   int32  
 2   path_id  int32  
 3   seg_id   int64  
 4   t1       int32  
 5   t2       int64  
 6   pp_id2   int64  
 7   time     int64  
 8   dis      float64
dtypes: float64(1), int32(4), int64(4)
memory usage: 1.5 GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30298852 entries, 0 to 30298851
Data columns (total 11 columns):
 #   Column   Dtype  
---  ------   -----  
 0   rid      int

InvalidIndexError: Reindexing only valid with uniquely valued Index objects